### Predict price using neural network

#### We have 3 files
-- imports-85.data ## This file have all the data in csv format<br>-- imports-85.names ## This file have all column names, description and range of values<br>
-- misc
<br><br>
Data is taken from the UCI site. <br>
Site url: https://archive.ics.uci.edu/ml/datasets/Automobile

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [2]:
# all column name list
column_names = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

In [3]:
# import data from csv
df = pd.read_csv("imports-85.data", names=column_names, header=None)

In [4]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [5]:
# All the nan values are given as '?'. So replaced all the '?' with nan
df[column_names] = df[column_names].replace({'?':np.nan})

In [6]:
df.isna().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

In [7]:
#Price is dependent attribute so removing all the records which have nan price
df.dropna(subset = ["price"], inplace=True)

In [8]:
#filled all the nan field with ffill
# most of the numerical fields are casted as object because of '?' so can't replace with mean and median
df.fillna(method='ffill', inplace=True)

In [9]:
# first 3 records of normalized-losses was nan so it was not replaced in ffill, so replaced with 164
df['normalized-losses'].fillna(164, inplace=True)

In [10]:
# Updating the data type of all the fields
convert_dict = {'symboling': int, 
                'normalized-losses': int,
                'make': object,
                'fuel-type': object,
                'aspiration': object,
                'num-of-doors': object,
                'body-style': object,
                'drive-wheels': object,
                'engine-location': object,
                'wheel-base': float,
                'length': float,
                'width': float,
                'height': float,
                'curb-weight': int,
                'engine-type': object,
                'num-of-cylinders': object,
                'engine-size': int,
                'fuel-system': object,
                'bore': float,
                'stroke': float,
                'compression-ratio': float,
                'horsepower': int,
                'peak-rpm': int,
                'city-mpg': int,
                'highway-mpg': int,
                'price': float
            } 
df = df.astype(convert_dict)


In [11]:
# Replace number of door from categorical value to number(two, four)
#df['num-of-doors'].unique()
num_door_dict = {
    'two': 2,
    'four': 4
}
df['num-of-doors'] = df['num-of-doors'].map(num_door_dict)

In [12]:
# Replace number of Cylinder from categorical value to number
#'four', 'six', 'five', 'three', 'twelve', 'two', 'eight'
# df['num-of-cylinders'].unique()
num_of_cylinders_dict = {
    'four': 4,
    'six': 6,
    'five': 5,
    'three': 3,
    'twelve': 12,
    'two': 2,
    'eight': 8
}
df['num-of-cylinders'] = df['num-of-cylinders'].map(num_of_cylinders_dict)

In [13]:
# updated all categorical fields to number using one hot encoding for the below columns

# engine-location
#drive-wheels
#engine-type
#make
#aspiration
#body-style
#fuel-system
#fuel-type
for col_name in ['engine-location','drive-wheels', 'engine-type', 'make', 'aspiration', 'body-style', 'fuel-system', 'fuel-type']:
    df = pd.concat([df, pd.get_dummies(df[col_name])], axis=1)
    df.drop([col_name], axis='columns', inplace=True)

In [14]:
# Create train and target data from the data frame
target = df['price']
train = df.drop(['price'],axis = 1)

In [15]:
#Creating Neural network
model = Sequential()


# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               8704      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 173,569
Trainable params: 173,569
Non-trainable params: 0
_________________________________________________________________


In [16]:
# fit data
model.fit(train, target, epochs=500, batch_size=32, validation_split = 0.2)


Train on 160 samples, validate on 41 samples
Epoch 1/500
160/160 [==============================] - 5s 33ms/step - loss: 13091.5020 - mean_absolute_error: 13091.5020 - val_loss: 12415.8001 - val_mean_absolute_error: 12415.8008
Epoch 2/500
160/160 [==============================] - 0s 706us/step - loss: 12229.5125 - mean_absolute_error: 12229.5117 - val_loss: 10765.7818 - val_mean_absolute_error: 10765.7822
Epoch 3/500
160/160 [==============================] - 0s 810us/step - loss: 9586.3506 - mean_absolute_error: 9586.3506 - val_loss: 6017.9463 - val_mean_absolute_error: 6017.9463
Epoch 4/500
160/160 [==============================] - 0s 1ms/step - loss: 5654.5765 - mean_absolute_error: 5654.5767 - val_loss: 3857.1506 - val_mean_absolute_error: 3857.1506
Epoch 5/500
160/160 [==============================] - 0s 1ms/step - loss: 6711.5214 - mean_absolute_error: 6711.5210 - val_loss: 3125.3323 - val_mean_absolute_error: 3125.3323
Epoch 6/500
160/160 [==============================] - 0s

Epoch 47/500
160/160 [==============================] - 0s 2ms/step - loss: 2752.1832 - mean_absolute_error: 2752.1831 - val_loss: 1761.8363 - val_mean_absolute_error: 1761.8363
Epoch 48/500
160/160 [==============================] - 0s 3ms/step - loss: 2682.7738 - mean_absolute_error: 2682.7739 - val_loss: 1755.1903 - val_mean_absolute_error: 1755.1903
Epoch 49/500
160/160 [==============================] - 0s 2ms/step - loss: 2678.2194 - mean_absolute_error: 2678.2192 - val_loss: 1756.6926 - val_mean_absolute_error: 1756.6926
Epoch 50/500
160/160 [==============================] - 1s 4ms/step - loss: 2628.2903 - mean_absolute_error: 2628.2903 - val_loss: 1751.8884 - val_mean_absolute_error: 1751.8883
Epoch 51/500
160/160 [==============================] - 0s 3ms/step - loss: 2673.7365 - mean_absolute_error: 2673.7363 - val_loss: 1973.8224 - val_mean_absolute_error: 1973.8224
Epoch 52/500
160/160 [==============================] - 0s 2ms/step - loss: 2742.1852 - mean_absolute_error: 2

160/160 [==============================] - 0s 2ms/step - loss: 2560.6201 - mean_absolute_error: 2560.6201 - val_loss: 1766.3347 - val_mean_absolute_error: 1766.3346
Epoch 94/500
160/160 [==============================] - 0s 2ms/step - loss: 2495.2666 - mean_absolute_error: 2495.2666 - val_loss: 1785.9581 - val_mean_absolute_error: 1785.9581
Epoch 95/500
160/160 [==============================] - 0s 3ms/step - loss: 2497.0351 - mean_absolute_error: 2497.0352 - val_loss: 1777.7390 - val_mean_absolute_error: 1777.7389
Epoch 96/500
160/160 [==============================] - 0s 2ms/step - loss: 2495.7783 - mean_absolute_error: 2495.7781 - val_loss: 1756.1479 - val_mean_absolute_error: 1756.1478
Epoch 97/500
160/160 [==============================] - 0s 1ms/step - loss: 2490.3289 - mean_absolute_error: 2490.3291 - val_loss: 1775.5959 - val_mean_absolute_error: 1775.5958
Epoch 98/500
160/160 [==============================] - 0s 1ms/step - loss: 2506.2901 - mean_absolute_error: 2506.2900 - va

Epoch 139/500
160/160 [==============================] - 0s 2ms/step - loss: 2493.9074 - mean_absolute_error: 2493.9072 - val_loss: 1990.3075 - val_mean_absolute_error: 1990.3075
Epoch 140/500
160/160 [==============================] - 0s 2ms/step - loss: 2474.8208 - mean_absolute_error: 2474.8208 - val_loss: 1865.6682 - val_mean_absolute_error: 1865.6682
Epoch 141/500
160/160 [==============================] - 0s 1ms/step - loss: 2438.0239 - mean_absolute_error: 2438.0239 - val_loss: 1845.2788 - val_mean_absolute_error: 1845.2788
Epoch 142/500
160/160 [==============================] - 0s 1ms/step - loss: 2383.9569 - mean_absolute_error: 2383.9568 - val_loss: 1803.0234 - val_mean_absolute_error: 1803.0232
Epoch 143/500
160/160 [==============================] - 0s 1ms/step - loss: 2403.7042 - mean_absolute_error: 2403.7043 - val_loss: 1863.7524 - val_mean_absolute_error: 1863.7523
Epoch 144/500
160/160 [==============================] - 0s 1ms/step - loss: 2366.1613 - mean_absolute_er

Epoch 185/500
160/160 [==============================] - 0s 2ms/step - loss: 2341.2551 - mean_absolute_error: 2341.2551 - val_loss: 1963.9043 - val_mean_absolute_error: 1963.9043
Epoch 186/500
160/160 [==============================] - 0s 1ms/step - loss: 2335.1456 - mean_absolute_error: 2335.1455 - val_loss: 1927.0340 - val_mean_absolute_error: 1927.0339
Epoch 187/500
160/160 [==============================] - 0s 1ms/step - loss: 2316.9697 - mean_absolute_error: 2316.9697 - val_loss: 1994.7731 - val_mean_absolute_error: 1994.7731
Epoch 188/500
160/160 [==============================] - 0s 2ms/step - loss: 2260.5619 - mean_absolute_error: 2260.5620 - val_loss: 1941.9200 - val_mean_absolute_error: 1941.9199
Epoch 189/500
160/160 [==============================] - 0s 1ms/step - loss: 2300.6365 - mean_absolute_error: 2300.6365 - val_loss: 1986.3656 - val_mean_absolute_error: 1986.3657
Epoch 190/500
160/160 [==============================] - 0s 513us/step - loss: 2305.8147 - mean_absolute_

...

/Users/positra/anaconda/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.104580). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


160/160 [==============================] - 1s 4ms/step - loss: 1968.8611 - mean_absolute_error: 1968.8610 - val_loss: 2283.8405 - val_mean_absolute_error: 2283.8406
Epoch 356/500
160/160 [==============================] - 0s 3ms/step - loss: 1852.5206 - mean_absolute_error: 1852.5208 - val_loss: 2315.5678 - val_mean_absolute_error: 2315.5679
Epoch 357/500
160/160 [==============================] - 1s 3ms/step - loss: 1868.7744 - mean_absolute_error: 1868.7744 - val_loss: 2473.3858 - val_mean_absolute_error: 2473.3857
Epoch 358/500
160/160 [==============================] - 1s 4ms/step - loss: 1889.7116 - mean_absolute_error: 1889.7117 - val_loss: 2557.8111 - val_mean_absolute_error: 2557.8110
Epoch 359/500
160/160 [==============================] - 0s 3ms/step - loss: 1781.1722 - mean_absolute_error: 1781.1722 - val_loss: 2466.2560 - val_mean_absolute_error: 2466.2561
Epoch 360/500
160/160 [==============================] - 0s 2ms/step - loss: 1777.6563 - mean_absolute_error: 1777.6565

Epoch 401/500
160/160 [==============================] - 0s 1ms/step - loss: 1527.8908 - mean_absolute_error: 1527.8909 - val_loss: 2322.7945 - val_mean_absolute_error: 2322.7947
Epoch 402/500
160/160 [==============================] - 0s 1ms/step - loss: 1765.8937 - mean_absolute_error: 1765.8938 - val_loss: 2249.5253 - val_mean_absolute_error: 2249.5254
Epoch 403/500
160/160 [==============================] - 0s 965us/step - loss: 1721.2285 - mean_absolute_error: 1721.2285 - val_loss: 2492.4724 - val_mean_absolute_error: 2492.4724
Epoch 404/500
160/160 [==============================] - 0s 1ms/step - loss: 1733.5669 - mean_absolute_error: 1733.5668 - val_loss: 2439.8202 - val_mean_absolute_error: 2439.8201
Epoch 405/500
160/160 [==============================] - 0s 1ms/step - loss: 1879.3008 - mean_absolute_error: 1879.3008 - val_loss: 2414.3145 - val_mean_absolute_error: 2414.3145
Epoch 406/500
160/160 [==============================] - 0s 1ms/step - loss: 1564.8586 - mean_absolute_

Epoch 447/500
160/160 [==============================] - 0s 1ms/step - loss: 1391.3309 - mean_absolute_error: 1391.3311 - val_loss: 2323.1570 - val_mean_absolute_error: 2323.1570
Epoch 448/500
160/160 [==============================] - 0s 1ms/step - loss: 1383.8214 - mean_absolute_error: 1383.8215 - val_loss: 2520.9850 - val_mean_absolute_error: 2520.9851
Epoch 449/500
160/160 [==============================] - 0s 1ms/step - loss: 1411.1469 - mean_absolute_error: 1411.1469 - val_loss: 2423.9991 - val_mean_absolute_error: 2423.9993
Epoch 450/500
160/160 [==============================] - 0s 684us/step - loss: 1400.4051 - mean_absolute_error: 1400.4050 - val_loss: 2477.1855 - val_mean_absolute_error: 2477.1855
Epoch 451/500
160/160 [==============================] - 0s 3ms/step - loss: 1423.3843 - mean_absolute_error: 1423.3843 - val_loss: 2497.5611 - val_mean_absolute_error: 2497.5610
Epoch 452/500
160/160 [==============================] - 0s 2ms/step - loss: 1449.0991 - mean_absolute_

Epoch 493/500
160/160 [==============================] - 0s 1ms/step - loss: 1413.3304 - mean_absolute_error: 1413.3304 - val_loss: 2233.8293 - val_mean_absolute_error: 2233.8293
Epoch 494/500
160/160 [==============================] - 0s 1ms/step - loss: 1405.2234 - mean_absolute_error: 1405.2234 - val_loss: 2236.8065 - val_mean_absolute_error: 2236.8064
Epoch 495/500
160/160 [==============================] - 0s 1ms/step - loss: 1435.2371 - mean_absolute_error: 1435.2371 - val_loss: 2269.6168 - val_mean_absolute_error: 2269.6167
Epoch 496/500
160/160 [==============================] - 0s 1ms/step - loss: 1281.4052 - mean_absolute_error: 1281.4053 - val_loss: 2383.2811 - val_mean_absolute_error: 2383.2810
Epoch 497/500
160/160 [==============================] - 0s 1ms/step - loss: 1277.6040 - mean_absolute_error: 1277.6039 - val_loss: 2198.8981 - val_mean_absolute_error: 2198.8979
Epoch 498/500
160/160 [==============================] - 0s 1ms/step - loss: 1287.5346 - mean_absolute_er

In [17]:
model.predict(train.iloc[[101]])

array([[18352.127]], dtype=float32)

In [18]:
target.iloc[[101]]

104    17199.0
Name: price, dtype: float64